In [ ]:
## reference https://huggingface.co/learn/nlp-course/en/chapter7/6?fw=pt#training-a-causal-language-model-from-scratch

In [1]:
#installing some libraries
!pip install datasets
!pip install --upgrade jupyter ipywidgets
!pip install bitsandbytes
!pip install accelerate



Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import torch, transformers
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from transformers import PreTrainedModel, PretrainedConfig
from transformers import AutoModel, AutoConfig,AutoModelForCausalLM, AutoConfig
from accelerate import Accelerator



import torch.nn as nn
import torch.nn.functional as F
#from datasets import load_dataset
import pandas as pd, numpy as np
from torch import cuda
import datetime
import warnings,itertools
from torch.optim.lr_scheduler import LambdaLR
from torch.cuda.amp import autocast, GradScaler

# Ignore all warnings
warnings.filterwarnings('ignore')
#pip install transformers bitsandbytes>=0.39.0 -q
import zipfile,logging

In [3]:
# global params for training

B,T = 8,1024
epoch = 200
random_init_wts = False
min_text_len = 0
# train_loss_list = []
# val_loss_list =[]
if cuda.is_available():
    device = torch.device('cuda:0')
    print(device)
else:
    device = 'cpu'
#print(device)
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"
#os.environ["MKL_DEBUG_CPU_TYPE"] = "5"
context_length = None
global_tr_loss = torch.inf
global_val_loss = torch.inf
#print(global_tr_loss)
model_path = os.path.join("model")
print(model_path)

cuda:0
model


In [4]:
directory = os.path.join('.','data','unzip_text_10M')  # Replace with your directory path
directory

'./data/unzip_text_10M'

In [5]:
def read_text(directory):
    directory = os.path.join('.','data','unzip_text_10M',str(directory))  # Replace with your directory path
    print(f"directory :{directory}")
    # List all files in the directory
    files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    print(f"files:{files}")
    text_content = []
    # Read each file
    total_lines = 0
    for filenum,filename in enumerate(files):
        file_path = os.path.join(directory, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            #first_line = file.read()
            #print(f"filename :{filename}->first few lines {first_line}")
            #continue
            #lines_list = [line.strip() for line in open(file_path, 'r')]
            text = file.read()
            text_content.append(text)
            print(f"the file:{filename} has been appeneded to the uber list and its length is {len(text_content)} ")
            #total_lines+=len(lines_list)
            #text_content.append(lines_list)
    
    flattened_list = ''.join(text_content)
    assert (len(flattened_list) == total_lines , f"Expected {len(flattened_list)} to be equal to {total_lines}" )
    
    return flattened_list

    

In [6]:
train_list = read_text("train_10M")
#print(train_dict)
#val_list = read_text("dev")

directory :./data/unzip_text_10M/train_10M
files:['switchboard.train', 'simple_wiki.train', 'open_subtitles.train', 'gutenberg.train', 'childes.train', 'bnc_spoken.train']
the file:switchboard.train has been appeneded to the uber list and its length is 1 
the file:simple_wiki.train has been appeneded to the uber list and its length is 2 
the file:open_subtitles.train has been appeneded to the uber list and its length is 3 
the file:gutenberg.train has been appeneded to the uber list and its length is 4 
the file:childes.train has been appeneded to the uber list and its length is 5 
the file:bnc_spoken.train has been appeneded to the uber list and its length is 6 


In [7]:
len(train_list)

54215049

In [8]:
chunks = len(train_list)//(B*T)
print(chunks)

6618


In [21]:
B*T

8192

In [28]:
tokenizer = AutoTokenizer.from_pretrained("gpt2-large",return_tensors = "pt" , truncate = True, return_overflowing_tokens=True , padding = False,)
enc_train = tokenizer(train_list[:50000*3])


Token indices sequence length is longer than the specified maximum sequence length for this model (51426 > 1024). Running this sequence through the model will result in indexing errors


In [29]:
comp_ratio = len(train_list)/len(enc_train['input_ids'])
comp_ratio

1054.2342200443356

In [30]:
val_list = read_text("dev")
enc_val = tokenizer(val_list[:20000*5])

directory :./data/unzip_text_10M/dev
files:['switchboard.dev', 'simple_wiki.dev', 'open_subtitles.dev', 'gutenberg.dev', 'childes.dev', 'bnc_spoken.dev']
the file:switchboard.dev has been appeneded to the uber list and its length is 1 
the file:simple_wiki.dev has been appeneded to the uber list and its length is 2 
the file:open_subtitles.dev has been appeneded to the uber list and its length is 3 
the file:gutenberg.dev has been appeneded to the uber list and its length is 4 
the file:childes.dev has been appeneded to the uber list and its length is 5 
the file:bnc_spoken.dev has been appeneded to the uber list and its length is 6 


In [31]:
def get_df_from_list(enc , B= B, T = T, val= False):
    chunk_size = B*T
    if val:
        chunk_size = 2*B*T
        
    long_list_inp = enc['input_ids']
    long_list_attention = enc['attention_mask']

    # Step 3: Split the list into chunks and pad the last chunk if necessary
    chunks_inp = [long_list_inp[i:i + chunk_size] for i in range(0, len(long_list_inp), chunk_size)]
    chunks_att = [long_list_attention[i:i + chunk_size] for i in range(0, len(long_list_attention), chunk_size)]
    df = pd.DataFrame({'input_ids': chunks_inp,'attention_mask':chunks_att})
    return df
    

In [32]:
df_train_temp = get_df_from_list(enc_train)
# Display the DataFrame
df_train_temp.head()
print(f"Length of the dataframe is = {len(df_train_temp)}")

Length of the dataframe is = 7


In [33]:
df_val_temp = get_df_from_list(enc_val , val = True)
# Display the DataFrame
df_val_temp.head()
print(f"Length of the VALDATION dataframe is = {len(df_val_temp)}")


Length of the VALDATION dataframe is = 3


In [34]:
def pad_df(df,eos_char = tokenizer.eos_token_id,B = B, T = T,val = False):
    if val:
        B = 2*B
    for ind,row in df.iterrows():
        if len(row['input_ids']) != B*T :
            print(f"row = {ind} and input_id length = {len(row['input_ids'])}")
            print(f"row = {ind} and attention length = {len(row['attention_mask'])}")
            pad_len = B*T - len(row['input_ids'])
            print(f"padding the row index {ind} with {pad_len} character")
            row['input_ids'] = row['input_ids']+ [eos_char] * pad_len
            #attention mask should be padded to 0
            row['attention_mask'] = row['attention_mask']+ [0] * pad_len
            print("#### POST CONCAT####")
            print(f"row = {ind} and input_id length = {len(row['input_ids'])}")
            print(f"row = {ind} and attention length ={len(row['attention_mask'])}")
    return df


def verify_len(df,val = False, B= B):
    row_ind = []
    if val:
        B = 2*B
    for ind,row in df.iterrows():
        if len(row['input_ids']) != B*T :
            row_ind.append(ind)
        else:
            continue
    if len(row_ind) !=0:
        print("CONCATENATION Did not work")
    else:
        print("CONCATENATION worked")
        
            
        
    
        
        
   
   

In [35]:
df_train  = pad_df(df_train_temp)
verify_len(df_train)

row = 6 and input_id length = 2274
row = 6 and attention length = 2274
padding the row index 6 with 5918 character
#### POST CONCAT####
row = 6 and input_id length = 8192
row = 6 and attention length =8192
CONCATENATION worked


In [36]:
df_val  = pad_df(df_val_temp, val = True)
verify_len(df_val, val = True)

row = 2 and input_id length = 1980
row = 2 and attention length = 1980
padding the row index 2 with 14404 character
#### POST CONCAT####
row = 2 and input_id length = 16384
row = 2 and attention length =16384
CONCATENATION worked


In [37]:
df_train.head()

,input_ids,attention_mask
0,"[32, 25, 197, 40, 1101, 1654, 484, 389, 13, 19...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,"[4398, 470, 1775, 326, 13, 198, 32, 25, 197, 4...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,"[290, 1243, 588, 326, 198, 33, 25, 197, 392, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,"[198, 33, 25, 197, 5779, 11, 1682, 11, 21480, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,"[6510, 11, 14104, 290, 27913, 13, 198, 32, 25,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


## Define the model

In [38]:
!pip install peft

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [39]:
from peft import LoraConfig, TaskType
target_modules = ["c_attn","q_proj", "k_proj", "v_proj", "out_proj", "fc_in", "fc_out", "wte"]
peft_config = LoraConfig(r=4, lora_alpha=16, target_modules=target_modules, lora_dropout=0.1, bias="none", task_type="CAUSAL_LM")


In [40]:
# Test the tokenizer:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from peft import get_peft_model

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=False,
)


model_4bit = AutoModelForCausalLM.from_pretrained(
    "gpt2-large",
    quantization_config=bnb_config, 
    )
#model_4bit_c = torch.compile(model_4bit)
model_4bit = get_peft_model(model_4bit, peft_config)
model_4bit.print_trainable_parameters()

`low_cpu_mem_usage` was None, now set to True since model is quantized.


trainable params: 943,428 || all params: 774,973,508 || trainable%: 0.1217


In [41]:


accelerator = Accelerator()


In [42]:
print(f"Model size is {model_4bit.get_memory_footprint()/1e6}MB")

Model size is 527.487184MB


### Data loaders and Dataset for batched training

In [43]:
class dataset_pyt_val(Dataset):
    def __init__(self, df, B = B, T = T ):
        self.df = df
        print(f"Value of B {B}")
                                        
    def __getitem__(self, idx):
        #print(f"inside loader...idx ->{idx}")
        input_id_temp = torch.tensor(self.df.iloc[idx]['input_ids'],dtype = torch.long)
        #print(f"length of text ->{len(text)}")
        #print(f"text ->{text}")
        #encodings = tokenizer(text, truncation=True, max_length= self.max_length, return_overflowing_tokens=True, padding = 'max_length',return_tensors='pt')
        att_mask = torch.tensor(self.df.iloc[idx]['attention_mask'],dtype = torch.long)
        
        
        # check the length of the encoded list
        
        #x_dict['input_id'] = input_ids_list
        #x_dict['attention_mask'] = input_ids_list
                
        #print(f"x_dict = {x_dict}")             
#       print(f"inside the loader and input_id = {input_ids} and its shape is {input_ids.shape}")
        #labels = input_id_list
        #input_ids = torch.tensor(input_id_list)
        #attention_mask = torch.tensor(attention_mask_list)
        #labels = torch.tensor(labels)
        #print(f"inside the loader and input_id shape= {input_ids.shape} attention_mask_shape is {attention_mask.shape} and label shape is {labels.shape}")
        #print(f"encoding = {encodings}")
        input_id =   input_id_temp.view(B,T)    
        attention_mask = att_mask.view(B,T)   
        
        return input_id, attention_mask
        
    def __len__(self):
        #return the length of the dataframe
        return len(self.df)

In [44]:
class dataset_pyt_train(Dataset):
    def __init__(self, df, B = B, T = T ):
        self.df = df
                                        
    def __getitem__(self, idx):
        #print(f"inside loader...idx ->{idx}")
        input_id_temp = torch.tensor(self.df.iloc[idx]['input_ids'],dtype = torch.long)
        #input_id_temp = input_id_temp.to(torch.int)
        
        #print(f"length of text ->{len(text)}")
        #print(f"text ->{text}")
        #encodings = tokenizer(text, truncation=True, max_length= self.max_length, return_overflowing_tokens=True, padding = 'max_length',return_tensors='pt')
        att_mask = torch.tensor(self.df.iloc[idx]['attention_mask'],dtype = torch.long)
        #att_mask = att_mask.to(torch.int)
        
        # check the length of the encoded list
        
        #x_dict['input_id'] = input_ids_list
        #x_dict['attention_mask'] = input_ids_list
                
        #print(f"x_dict = {x_dict}")             
#       print(f"inside the loader and input_id = {input_ids} and its shape is {input_ids.shape}")
        #labels = input_id_list
        #input_ids = torch.tensor(input_id_list)
        #attention_mask = torch.tensor(attention_mask_list)
        #labels = torch.tensor(labels)
        #print(f"inside the loader and input_id shape= {input_ids.shape} attention_mask_shape is {attention_mask.shape} and label shape is {labels.shape}")
        #print(f"encoding = {encodings}")
        input_id =   input_id_temp.view(B,T)    
        attention_mask = att_mask.view(B,T)   
        
        return input_id, attention_mask
        
    def __len__(self):
        #return the length of the dataframe
        return len(self.df)

In [45]:
#train_dataset = dataset_pyt(filtered_df,tokenizer = tokenizer)
train_dataset = dataset_pyt_train(df_train)
val_dataset = dataset_pyt_val(df_val)
#test_dataset = dataset_pyt(df_test,tokenizer = tokenizer)

train_loader = DataLoader(train_dataset,batch_size = 1, shuffle = True , num_workers = 4, pin_memory = True)
val_loader = DataLoader(val_dataset,batch_size = 1, shuffle = True , num_workers = 4, pin_memory = True)
#test_loader = DataLoader(test_dataset,batch_size = batch_size, shuffle = False, collate_fn = custom_collate_fn)


Value of B 8


In [28]:
# inp1 = torch.randint(low=0, high=50257, size=(B, T), dtype=torch.long)
# inp1.shape
# att1 = torch.randint(low=0, high=1, size=(B, T), dtype=torch.long)

## Trainer

In [46]:
output_dir = os.path.join(".","model")

In [29]:
# inp , att = train_dataset[0]
# inp_g = inp.to(device)
# att_g = att.to(device)

# print(inp_g.dtype,att_g.dtype)
# print(inp_g)


In [30]:
# model_o = model_4bit(input_ids = inp_g ,attention_mask = att_g, labels = inp_g)

In [31]:
print(f"Length of the train loader is {len(train_loader)}")
print(f"Length of the val loader is {len(val_loader)}")
print(f"num_tokens= {B*T*len(train_loader)}")

Length of the train loader is 2099
Length of the val loader is 1063
num_tokens= 17195008


In [32]:
#model_output = model(input_ids = inp ,attention_mask = att, labels = inp)

In [33]:
def write_file(log_message, model_name = "GPT2-L" ,random_init_wts = random_init_wts ):
    current_datetime = datetime.datetime.now()
    # Extract date and time components
    current_date = str(current_datetime.date())
    log_file = model_name +'_'+'Quantized'+ '_'+current_date+'.log'
    print(f"*****LOGGING INFO IN {log_file}*********")
    filepath = os.path.join("model",log_file)
    logging.basicConfig(filename=filepath, 
                    filemode='a',  # Overwrite the log file each time
                    format='%(asctime)s - %(levelname)s - %(message)s', 
                    level=logging.DEBUG)
    logger = logging.getLogger()
    logger.info(log_message)
    
    

    
    

In [34]:
@torch.no_grad
def eval_model(val_loader, model, epoch , device = device,tokenizer = tokenizer):
    global global_val_loss
    #m = nn.Softmax()
    model.eval()
    model.to(device)
    e = epoch+1
    val_loss_accum = 0.0
    #criterion = torch.nn.BCEWithLogitsLoss()
    print(f"inside validation data for epoch {e}")
    #y_hat_val_list = []
    #y_val_list = []
    with torch.no_grad():
        for ind,(input_id,attention_mask) in enumerate(val_loader):
        #print(f"id_list{id_list}")
            ids = input_id.to(device=device, non_blocking=True)
            att_mask = attention_mask.to(device=device, non_blocking=True)
            labels = ids
        #predictions
        #print(f"input_ids device = {input_ids.device}")
            #with autocast(dtype = torch.bfloat16):
            model_output = model(input_ids = ids ,attention_mask = att_mask, labels = labels)
            act_loss = model_output.loss
        
        val_loss_accum+=act_loss.detach().item()
        del ids,att_mask,labels,act_loss
    
    if val_loss_accum < global_val_loss:
        print(f"Val loss has decreased -->reducing the global validation loss from {global_val_loss:.2f} to {val_loss_accum:.2f}")
        s1 = f"Val loss has decreased -->reducing the global validation loss from {global_val_loss:.2f} to {val_loss_accum:.2f}"
        global_val_loss = val_loss_accum
        print(f" validation loss for epoch = {e} is {val_loss_accum:.4f}")
        s2 = f" validation loss for epoch = {e} is {val_loss_accum:.4f}"
        #print metrics and save the model
        #y_hat_val = torch.cat(y_hat_val_list)
        #y_val = torch.cat(y_val_list)
        #acc_val = accuracy_score(y_val.cpu().numpy(), y_hat_val.cpu().numpy())
        #f1_val = f1_score(y_val.cpu().numpy(), y_hat_val.cpu().numpy(), average='micro')
        print(f" epoch= {e} :  val loss is {val_loss_accum:.4f} ")
        s3 = f" epoch= {e} :  val loss is {val_loss_accum:.4f} "
        #save the model
        
        # Get the current date and time
        current_datetime = datetime.datetime.now()
        # Extract date and time components
        current_date = str(current_datetime.date())
        current_time = str(current_datetime.time()).split('.')[0]
        file_name = 'model'+ current_date+current_time+'.pth'
        path = os.path.join("model",file_name)
        print(f"saving the model {file_name}")
        s4 = f"saving the model {file_name}"
        #torch.save(model.state_dict(), path)
        model.save_pretrained(path)
        tokenizer.save_pretrained(path)
        log_message = s1+s2+s3+s4
        write_file(log_message)
        
        #plot_confusion_matrix(y_val.cpu().numpy(), y_hat_val.cpu().numpy(), labels)
    else:
        print(f"No improvement in validation loss-->epoch= {e} and global val loss is {global_val_loss:.2f}")
        
    
    
    


In [35]:
def train_model(train_loader,val_loader,model,num_epoch = 200,device = device,tokenizer = tokenizer):
    global global_tr_loss
    model.train()
    device = device
    lr_custom = 2e-4
    print(f"inside train model. Device = {device}")
    #adding betas params per the paper
    #model.to(device)
    optimizer = torch.optim.AdamW(params =  model.parameters(), lr= lr_custom,betas = (.9,.95) ,weight_decay = .1)
    total_batch_size = 2**19
    grad_accum_step = total_batch_size//(B*T)
    from accelerate import Accelerator

    
    extra_train = .1*num_epoch
    #m = nn.Softmax()
    max_train_steps = int(num_epoch +extra_train )
    import time
    from transformers import get_linear_schedule_with_warmup
    scheduler_cos = transformers.get_cosine_schedule_with_warmup( optimizer= optimizer, num_warmup_steps =num_epoch*.1 ,num_training_steps= num_epoch-1 ,last_epoch = -1 )
    model, optimizer, train_loader, val_loader, lr_scheduler = accelerator.prepare( model, optimizer, train_loader, val_loader, scheduler_cos)
        
    for i in range (max_train_steps):
        epoch_start_time = time.time()
        epoch_train_loss = 0.0
        optimizer.zero_grad(set_to_none=True)
        batch_loss = 0.0
        for ind,(input_id,attention_mask) in enumerate(train_loader):
            if ind == int(len(train_loader)/2):
                batch_time = time.time()
                duration = batch_time - epoch_start_time
                print(f"executing epoch:{i+1}, it took {duration/60} mins from beginning of epoch till batch#{ind}")
            
            ids = input_id.to(device=device, non_blocking=True)
            att_mask = attention_mask.to(device=device, non_blocking=True)
            labels = ids
            
            #with autocast(dtype = torch.bfloat16):
            model_output = model(input_ids = ids ,attention_mask = att_mask, labels = labels)
            act_loss = model_output.loss
            #print(f"inside batch_processing. ind = {ind}|loss for ind = {act_loss} |batch_loss ={batch_loss} | epoch_train_loss = {epoch_train_loss}")
            batch_loss +=act_loss.detach().item()
            #print(f"incrementing batch_loss. ind = {ind}|loss for ind = {act_loss} |batch_loss ={batch_loss} | epoch_train_loss = {epoch_train_loss}")
            act_loss = act_loss/grad_accum_step
            accelerator.backward(act_loss)
            epoch_train_loss += act_loss.detach().item()
            
            if (ind + 1)% grad_accum_step == 0:
                #print(f"Gradient accum step = {ind}|batch_loss before averaging ={batch_loss} | epoch_train_loss = {epoch_train_loss}")
                norm = torch.nn.utils.clip_grad_norm(model.parameters() , 1.0)
                if i <= num_epoch:
                    optimizer.step()
                    optimizer.zero_grad(set_to_none=True)
                else:
                    optimizer_reduced_lr.step()
                    optimizer_reduced_lr.zero_grad(set_to_none=True)
                #print(f"Gradient accum step = {ind}|batch_loss after averaging ={batch_loss} | epoch_train_loss = {epoch_train_loss}")
                #print(f"Gradient accum step = {ind}|batch_loss after averaging ={batch_loss} | epoch_train_loss = {epoch_train_loss}")
                                
            del ids,att_mask,labels
            
        
        
        #batch processing complete 
        if i <= num_epoch:
            current_lr = scheduler_cos.get_last_lr()[0]
            scheduler_cos.step()
        if i >= num_epoch:
            optimizer_reduced_lr = torch.optim.AdamW(params =  model.parameters(), lr= current_lr ,betas = (.9,.95), weight_decay=.1)
            scheduler_constant = transformers.get_constant_schedule_with_warmup( optimizer = optimizer_reduced_lr ,num_warmup_steps = 0, last_epoch = -1 )
            model, optimizer, train_dataloader, eval_dataloader, lr_scheduler = accelerator.prepare( model, optimizer_reduced_lr, train_dataloader, val_dataloader, scheduler_constant)
            optimizer_reduced_lr.zero_grad(set_to_none=True)
            
            scheduler_constant.step()
                      
        #mean_loss = torch.mean(torch.tensor(epoch_train_loss))
        epoch_end_time = time.time()
        epoch_durn = (epoch_end_time - epoch_start_time)
        num_token = B*T*len(train_loader)
        if (epoch_train_loss < global_tr_loss) :
            print(f"training loss has decreased---> reducing the global loss from {global_tr_loss:.2f} to {epoch_train_loss:.2f} | throughput = {int(num_token/epoch_durn)} tokens/second | norm = {norm:.4f} | learning rate = {current_lr:.5e}")
            global_tr_loss = epoch_train_loss
            print(f" epoch= {i+1} and  train loss is {epoch_train_loss:.2f}")
            #printing training metrices
            #y_hat = torch.cat(y_hat_list)
            #y = torch.cat(label_list)
            #acc = accuracy_score(y.cpu().numpy(), y_hat.cpu().numpy())
            #f1 = f1_score(y.cpu().numpy(), y_hat.cpu().numpy(), average='micro')
            #checking validation metrices
            if (i%4 == 0):
                eval_model(val_loader, model, epoch = i , device = device,tokenizer = tokenizer)
            
        else:
            print(f"No improvement in training loss..the global training loss is -->{global_tr_loss:.2f} ")
            print(f" epoch= {i+1} and mean train loss is {epoch_train_loss:.2f}")
        
        
    
    return model
        
            
            
    
    

In [36]:
tr_model = train_model(train_loader, val_loader, model =  model_4bit,tokenizer = tokenizer)

inside train model. Device = cuda:0
executing epoch:1, it took 11.394673649470011 mins from beginning of epoch till batch#1049
No improvement in training loss..the global training loss is -->inf 
 epoch= 1 and mean train loss is nan
executing epoch:2, it took 11.377593251069387 mins from beginning of epoch till batch#1049
No improvement in training loss..the global training loss is -->inf 
 epoch= 2 and mean train loss is nan
executing epoch:3, it took 11.383647147814433 mins from beginning of epoch till batch#1049
No improvement in training loss..the global training loss is -->inf 
 epoch= 3 and mean train loss is nan
executing epoch:4, it took 11.390172771612804 mins from beginning of epoch till batch#1049
No improvement in training loss..the global training loss is -->inf 
 epoch= 4 and mean train loss is nan
executing epoch:5, it took 11.405706361929576 mins from beginning of epoch till batch#1049


KeyboardInterrupt: 

## Use this section if you want a model with Random weights

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
# Define the model name
model_name = "distilgpt2"
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Load the model configuration
config = AutoConfig.from_pretrained(model_name)

# Initialize the model with random weights
model = AutoModelForCausalLM.from_config(config)

# Check the model
#print(model)



In [ ]:
x = torch.tensor(df_train.iloc[178]['input_ids'], dtype = torch.long)
att = torch.tensor(df_train.iloc[178]['attention_mask'], dtype = torch.long)
random_out = model(input_ids = x.view(B,T) , attention_mask = att.view(B,T), labels =   x.view(B,T) )

In [ ]:
random_out